In [11]:
import os

from magic_pdf.data.data_reader_writer import FileBasedDataWriter, FileBasedDataReader
from magic_pdf.data.dataset import PymuDocDataset
from magic_pdf.model.doc_analyze_by_custom_model import doc_analyze
from magic_pdf.config.enums import SupportedPdfParseMethod

# args
pdf_file_name = r"C:\Users\HP\Desktop\test3.pdf"  # replace with the real pdf path
name_without_suff = pdf_file_name.split(".")[0]

# prepare env
local_image_dir, local_md_dir = r"C:\Users\HP\Desktop\output_step3\images", r"C:\Users\HP\Desktop\output_step3\test3"
image_dir = str(os.path.basename(local_image_dir))

os.makedirs(local_image_dir, exist_ok=True)

image_writer, md_writer = FileBasedDataWriter(local_image_dir), FileBasedDataWriter(
    local_md_dir
)

# read bytes
reader1 = FileBasedDataReader("")
pdf_bytes = reader1.read(pdf_file_name)  # read the pdf content

# proc
## Create Dataset Instance
ds = PymuDocDataset(pdf_bytes)

## inference
if ds.classify() == SupportedPdfParseMethod.OCR:
    infer_result = ds.apply(doc_analyze, ocr=True)

    ## pipeline
    pipe_result = infer_result.pipe_ocr_mode(image_writer)

else:
    infer_result = ds.apply(doc_analyze, ocr=False)

    ## pipeline
    pipe_result = infer_result.pipe_txt_mode(image_writer)


### get model inference result
model_inference_result = infer_result.get_infer_res()



### get markdown content
md_content = pipe_result.get_markdown(image_dir)

### dump markdown
pipe_result.dump_md(md_writer, f"{name_without_suff}.md", image_dir)

### get content list content
content_list_content = pipe_result.get_content_list(image_dir)

### dump content list
pipe_result.dump_content_list(md_writer, f"{name_without_suff}_content_list.json", image_dir)


2025-02-17 15:23:21.989 | INFO     | magic_pdf.data.dataset:__init__:156 - lang: None
2025-02-17 15:23:24.540 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:57 - cid_count: 0, text_len: 4723, cid_chars_radio: 0.0
2025-02-17 15:23:27.830 | INFO     | magic_pdf.model.pdf_extract_kit:__call__:217 - layout detection time: 3.19
2025-02-17 15:23:32.152 | INFO     | magic_pdf.model.pdf_extract_kit:__call__:223 - mfd time: 4.32
2025-02-17 15:23:32.202 | INFO     | magic_pdf.model.pdf_extract_kit:__call__:230 - formula nums: 0, mfr time: 0.05
2025-02-17 15:23:35.251 | INFO     | magic_pdf.model.pdf_extract_kit:__call__:264 - det time: 3.05
2025-02-17 15:23:35.252 | INFO     | magic_pdf.model.pdf_extract_kit:__call__:304 - table time: 0.0
2025-02-17 15:23:35.252 | INFO     | magic_pdf.model.doc_analyze_by_custom_model:doc_analyze:236 - -----page_id : 0, page total time: 10.62-----
2025-02-17 15:23:37.450 | INFO     | magic_pdf.model.pdf_extract_kit:__call__:217 - layout detection tim

现在是就是一个pdf文件生成了md，然后他把表格和图片都放在C:\Users\HP\Desktop\output_step3\images，在原本的位置生成了占位符，然后就是说config_list也是有一些很有用的数据，数据类型，数据内容等等都是在这其中呈现，现在需要做的是找一个读图片和表格最好的模型把占位符换成文字代替。

In [12]:
import os
import base64
 
from openai import OpenAI
 
client = OpenAI(
    api_key="sk-3scaeuq5Pvut0jW1d7YsAQV3fkXr3X6rfwk61IEDzgqHUxhG",
    base_url="https://api.moonshot.cn/v1",
)
 
# 在这里，你需要将 kimi.png 文件替换为你想让 Kimi 识别的图片的地址
image_path = r"C:\Users\HP\Desktop\output_step3\images\7706d6ed5ddd1cef4b715ee682cf2eb945cab3d11fb97dbe7a869b16f0e7fda3.jpg"
 
with open(image_path, "rb") as f:
    image_data = f.read()
 
# 我们使用标准库 base64.b64encode 函数将图片编码成 base64 格式的 image_url
image_url = f"data:image/{os.path.splitext(image_path)[1]};base64,{base64.b64encode(image_data).decode('utf-8')}"
 
 
completion = client.chat.completions.create(
    model="moonshot-v1-128k-vision-preview",
    messages=[
        {"role": "system", "content": "你是 Kimi，由 Moonshot AI 提供的人工智能助手，你更擅长中文和英文的对话。你会为用户提供安全，有帮助，准确的回答。你在此对话中扮演的角色是图像分析器，请务必注意要给用户**完整描述**图片上的**所有内容**，包含位置关系，文字，分类，内容关系等。"},
        {
            "role": "user",
            # 注意这里，content 由原来的 str 类型变更为一个 list，这个 list 中包含多个部分的内容，图片（image_url）是一个部分（part），
            # 文字（text）是一个部分（part）
            "content": [
                {
                    "type": "image_url", # <-- 使用 image_url 类型来上传图片，内容为使用 base64 编码过的图片内容
                    "image_url": {
                        "url": image_url,
                    },
                },
                {
                    "type": "text",
                    "text": "请识别该图片，并描述这个图片。要求：这是一张布局图，你在描述图片时，请将重点放在**位置关系**和**风向标**上，且你需要**完整**地体现图中的**所有内容**。描述方式应该是类似：A在B的正下方紧邻，A在B的东偏北40°不远处，A和B的中间有C，A布置在C上（如果出现A区域被C区域包含），A和B相邻，A和B中间有过道等描述。如果存在同名的区域，请为它编号，比如A1，A2，草坪1，草坪2。注意：你在描述完成时应该检查每个区域的位置特征是否都被**完整体现**，比如A可能既布置在C上，又紧邻布置在B的下方，且同时与D相隔一个过道并在D的上方布置。请以左上角为基准开始描述。", # <-- 使用 text 类型来提供文字指令，例如“描述图片内容”
                },
            ],
        },
    ],
)
 
print(completion.choices[0].message.content)

这是一张布局图，图中包含多个区域和设施。以下是对图中内容的详细描述：

1. **变压器**位于图的左上角，布置在**草坪1**上。

2. **草坪1**位于变压器的右侧，与变压器相邻。

3. **草坪2**位于图的右上角，与**办公楼**相邻。

4. **办公楼**位于图的右中部，布置在草坪2的下方。

5. **职工食堂、职工宿舍**位于办公楼的正下方，紧邻办公楼。

6. **成品库房一**位于图的中上部，紧邻**发电机**。

7. **发电机**位于成品库房一的左侧，布置在**一车间**和**二车间**之间。

8. **一车间**位于图的左中部，紧邻发电机的左侧。

9. **二车间**位于一车间的正下方，紧邻一车间。

10. **锅炉房**位于图的左下角，布置在**草坪3**上。

11. **储罐区及应急池**位于锅炉房的正下方，紧邻锅炉房。

12. **草坪3**位于储罐区及应急池的右侧，与储罐区及应急池相邻。

13. **成品库房二**位于图的中下部，紧邻**三车间**。

14. **三车间**位于成品库房二的右侧，布置在**机修房**和**固废库房**之间。

15. **机修房**位于三车间的右侧，紧邻三车间。

16. **固废库房**位于三车间的正下方，紧邻三车间。

图中还包含一个风向标，位于图的右上角，指示风向为北偏东40°。


In [13]:
import re
import shutil
from pathlib import Path

def update_markdown_content(md_file_path, image_name, new_content):
    try:
        md_file_path = Path(md_file_path)
        
        # 读取Markdown文件
        with open(md_file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        
        # 创建备份
        backup_path = md_file_path.with_suffix(md_file_path.suffix + '.backup')
        shutil.copy2(md_file_path, backup_path)
        
        # 构建匹配模式 - 适配 ![](images/filename.jpg) 格式
        pattern = f"!\\[\\]\\(images/{re.escape(image_name)}\\)"
        if not re.search(pattern, content):
            raise ValueError(f"Image reference '{image_name}' not found in the Markdown file")
            
        updated_content = re.sub(pattern, new_content, content)
        
        # 写入更新后的内容
        with open(md_file_path, 'w', encoding='utf-8') as file:
            file.write(updated_content)
            
        print(f"Successfully updated {md_file_path}")
        print(f"Backup created at {backup_path}")
        
    except FileNotFoundError:
        print(f"Error: File not found - {md_file_path}")
    except Exception as e:
        print(f"Error occurred: {str(e)}")

if __name__ == "__main__":
    # 示例使用:
    md_file_path = r"C:\Users\HP\Desktop\test3.md"
    # 只需要文件名部分
    image_name = "7706d6ed5ddd1cef4b715ee682cf2eb945cab3d11fb97dbe7a869b16f0e7fda3.jpg"
    new_content = completion.choices[0].message.content
    
    update_markdown_content(md_file_path, image_name, new_content)


Successfully updated C:\Users\HP\Desktop\test3.md
Backup created at C:\Users\HP\Desktop\test3.md.backup
